In [1]:
!pip install --upgrade "google-cloud-aiplatform[langchain,reasoningengine]" cloudpickle==3.0.0 pydantic==2.7.4 langchain-google-community google-cloud-discoveryengine google-api-python-client requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-google-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 800.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 876.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 915.3 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
PROJECT_ID = "qwiklabs-gcp-02-a5369b2dfad3"  
LOCATION = "us-central1"
STAGING_BUCKET = "gs://qwiklabs-gcp-02-a5369b2dfad3"  

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [2]:
from googleapiclient import discovery
from IPython.display import display, Markdown
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain_core import prompts
from langchain.memory import ChatMessageHistory
from vertexai.preview import reasoning_engines

In [3]:
model = "gemini-1.5-pro-001"

In [10]:
DATA_STORE_ID = "cymbal-movie-data-store_1726607760029"
LOCATION_ID = "global"

In [11]:
def search_kaggle_movies(query: str) -> str:
    """Search across records in the Kaggle Movies data set."""
    from langchain_google_community import VertexAISearchRetriever

    retriever = VertexAISearchRetriever(
        project_id=PROJECT_ID,
        data_store_id=DATA_STORE_ID,
        location_id=LOCATION_ID,
        engine_data_type=1,
        max_documents=10,
    )

    result = str(retriever.invoke(query))
    return result

In [12]:
search_kaggle_movies("Harry Potter")

'[Document(metadata={\'id\': \'dd68385e2b3e0a0ddd497d5eb1a9dcc9\', \'name\': \'projects/573274662810/locations/global/collections/default_collection/dataStores/cymbal-movie-data-store_1726607760029/branches/0/documents/dd68385e2b3e0a0ddd497d5eb1a9dcc9\'}, page_content=\'{"vote_average": 7.4, "poster_path": "https://image.tmdb.org/t/p/original/5aOyriWkPec0zUDxmHFP9qMmBaj.jpg", "spoken_languages": "[{\\\'iso_639_1\\\': \\\'en\\\', \\\'name\\\': \\\'English\\\'}]", "overview": "Returning for his fifth year of study at Hogwarts, Harry is stunned to find that his warnings about the return of Lord Voldemort have been ignored. Left with no choice, Harry takes matters into his own hands, training a small group of students \\\\u2013 dubbed \\\'Dumbledore\\\'s Army\\\' \\\\u2013 to defend themselves against the dark arts.", "imdb_id": "tt0373889", "vote_count": "5633", "genres": "[{\\\'id\\\': 12, \\\'name\\\': \\\'Adventure\\\'}, {\\\'id\\\': 14, \\\'name\\\': \\\'Fantasy\\\'}, {\\\'id\\\': 107

In [13]:
prompt = {
    "history": lambda x: x["history"],
    "input": lambda x: x["input"],
    "agent_scratchpad": (lambda x: format_to_tool_messages(x["intermediate_steps"])),
} | prompts.ChatPromptTemplate.from_messages(
    [
        prompts.MessagesPlaceholder(variable_name="history"),
        ("user", "{input}"),
        prompts.MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Initialize session history
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [14]:
agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=model,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0},
    tools=[search_kaggle_movies],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [15]:
response = agent.query(
    input="List some sci-fi movies from the 1990s",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

Here are some sci-fi movies from the 1990s:

* The Time Machine (1960) A Victorian Englishman travels to the far future and finds that humanity has divided into two hostile species. 
* Metamorphosis: The Alien Factor (1990) A virus from outer space transforms a bio-researcher into a blood thirsty monster. 

In [16]:
agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=model,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0},
    tools=[search_kaggle_movies],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [17]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic==2.7.4",
        "langchain-google-community",
        "google-cloud-discoveryengine",
    ],
)

Creating bucket qwiklabs-gcp-02-a5369b2dfad3 in location='us-central1'
Writing to gs://qwiklabs-gcp-02-a5369b2dfad3/reasoning_engine/reasoning_engine.pkl
Writing to gs://qwiklabs-gcp-02-a5369b2dfad3/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://qwiklabs-gcp-02-a5369b2dfad3/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/573274662810/locations/us-central1/reasoningEngines/9176946257883561984/operations/8075995541185495040
ReasoningEngine created. Resource name: projects/573274662810/locations/us-central1/reasoningEngines/9176946257883561984
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/573274662810/locations/us-central1/reasoningEngines/9176946257883561984')


In [18]:
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=PROJECT_ID)
response = request.execute()
project_number = response["projectNumber"]
project_number

'573274662810'

In [19]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role=roles/discoveryengine.editor

I0000 00:00:1726608527.425875     479 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


Updated IAM policy for project [qwiklabs-gcp-02-a5369b2dfad3].
bindings:
- members:
  - serviceAccount:573274662810-compute@developer.gserviceaccount.com
  role: roles/aiplatform.admin
- members:
  - serviceAccount:service-573274662810@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-573274662810@gcp-sa-aiplatform-re.iam.gserviceaccount.com
  role: roles/aiplatform.reasoningEngineServiceAgent
- members:
  - serviceAccount:service-573274662810@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:573274662810-compute@developer.gserviceaccount.com
  role: roles/artifactregistry.admin
- members:
  - serviceAccount:573274662810-compute@developer.gserviceaccount.com
  - serviceAccount:qwiklabs-gcp-02-a5369b2dfad3@qwiklabs-gcp-02-a5369b2dfad3.iam.gserviceaccount.com
  role: roles/bigquery.admin
- members:
  - serviceAccount:573274662810@cloudbuild.gservice

In [25]:
response = remote_agent.query(
    input="List some sci-fi movies from the 1990s",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

Here are some sci-fi movies from the 1990s:

* **Science Fiction** (2002) Nine-year-old Andreas Decker is the new kid in town who claims his parents are aliens.
* **Moon 44** (1990) In 2038, Earth's resources are depleted, and corporations battle for minerals on other planets. 



In [26]:
response = remote_agent.query(
    input="Who are the actors in The Matrix?",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

The main actors in The Matrix (1999) are: 

* Keanu Reeves
* Laurence Fishburne 
* Carrie-Anne Moss
* Hugo Weaving 
* Joe Pantoliano 



In [27]:
response = remote_agent.query(
    input="Are those actors in any other movies?",
    config={"configurable": {"session_id": "demo"}},
)

display(Markdown(response["output"]))

Please clarify which actors you're interested in.  I listed two movies, but didn't mention any actors.  Can you tell me the names of the actors you'd like to know more about? 
